### Import Required Libraries and Set Up Environment Variables

In [5]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from datetime import datetime

# Load the NASA_API_KEY from the .env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')



### CME Data

In [6]:
import os
from dotenv import load_dotenv

# Load the NASA API key from .env file
load_dotenv()
api_key = os.getenv("NASA_API_KEY")

# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={api_key}"


In [7]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
import requests

cme_response = requests.get(cme_url)

In [8]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [9]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
import json

if cme_json:
# Print the first entry formatted in JSON if it's a non-empty list
    print(json.dumps(cme_json[0], indent=4))
else:
    print("No data found in the API response.")


{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [15]:
# Convert cme_json to a Pandas DataFrame 
import pandas as pd
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]
# Display the resulting DataFrame
print("CME DataFrame with selected columns:")
display(cme_df.head())

CME DataFrame with selected columns:


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [16]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])
print("\nCME DataFrame after dropping rows with missing 'linkedEvents':")
display(cme_df.head())


CME DataFrame after dropping rows with missing 'linkedEvents':


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [17]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for idx in cme_df.index:
    # Iterate over each dictionary in the list
    for event in cme_df.at[idx, 'linkedEvents']:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
       expanded_rows.append({
            'activityID': cme_df.at[idx, 'activityID'],
            'startTime': cme_df.at[idx, 'startTime'],
            'linkedEvent': event
        })
# Create a new DataFrame from the expanded rows
expanded_cme_df = pd.DataFrame(expanded_rows)


print("\nExpanded CME DataFrame with individual linkedEvents:")
display(expanded_cme_df.head())


Expanded CME DataFrame with individual linkedEvents:


,activityID,startTime,linkedEvent
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [19]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(linked_events_dict):
    try:
        # Extract the 'activityID' from the dictionary
        return linked_events_dict['activityID']
    except (TypeError, KeyError) as e:
        # Log the error or print it for debugging
        print(f"Error extracting activityID: {e}")
        return None
# Test the function using an example row from linkedEvents
example_activityID = extract_activityID_from_dict(expanded_cme_df['linkedEvent'][0])
print("Example extracted activityID:", example_activityID)


Example extracted activityID: 2013-05-04T04:52:00-IPS-001


In [20]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['linkedEvent'].apply(lambda x: extract_activityID_from_dict(x))


print("\nDataFrame after adding 'GST_ActivityID' column:")
from IPython.display import display
display(expanded_cme_df.head())


DataFrame after adding 'GST_ActivityID' column:


,activityID,startTime,linkedEvent,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [21]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs
expanded_cme_df = expanded_cme_df.dropna(subset=['GST_ActivityID'])

# Display the DataFrame to verify rows with missing GST_ActivityID were removed
print("\nDataFrame after dropping rows with missing 'GST_ActivityID':")
display(expanded_cme_df.head())


DataFrame after dropping rows with missing 'GST_ActivityID':


,activityID,startTime,linkedEvent,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [22]:
# Print out the datatype of each column in this DataFrame
print("\nData types of each column:")
print(expanded_cme_df.dtypes)


Data types of each column:
activityID        object
startTime         object
linkedEvent       object
GST_ActivityID    object
dtype: object


In [23]:
# Convert the 'GST_ActivityID' column to string format
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['GST_ActivityID'].astype(str)

# Convert startTime to datetime format
expanded_cme_df['startTime'] = pd.to_datetime(expanded_cme_df['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
expanded_cme_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop linkedEvents
expanded_cme_df = expanded_cme_df.drop(columns=['linkedEvent'])

# Verify that all steps were executed correctly
print("\nDataFrame after formatting columns and renaming:")
display(expanded_cme_df.head())


DataFrame after formatting columns and renaming:


,cmeID,startTime_CME,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01 03:12:00+00:00,2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03 22:36:00+00:00,2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09 19:29:00+00:00,2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T04:12:00-SEP-001


In [24]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.
gst_related_cmes = expanded_cme_df[expanded_cme_df['GST_ActivityID'].str.contains('GST', na=False)]

# Display the filtered DataFrame
print("\nFiltered DataFrame with GST-related CMEs only:")
display(gst_related_cmes.head())


Filtered DataFrame with GST-related CMEs only:


,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


### GST Data

In [25]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [26]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

In [27]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
import json
if gst_json:
    print(json.dumps(gst_json[0], indent=4))
else:
    print("No data found in the GST API response.")

{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [28]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: gstID, startTime, linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]


print("\nGST DataFrame with selected columns:")
from IPython.display import display
display(gst_df.head())


GST DataFrame with selected columns:


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [29]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=['linkedEvents'])


print("\nGST DataFrame after dropping rows with missing 'linkedEvents':")
from IPython.display import display
display(gst_df.head())


GST DataFrame after dropping rows with missing 'linkedEvents':


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [30]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True).dropna(subset=['linkedEvents'])


In [31]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer
gst_df['CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs
gst_df = gst_df.dropna(subset=['CME_ActivityID'])


print("\nGST DataFrame after adding 'CME_ActivityID' column and dropping rows with missing values:")
display(gst_df.head())


GST DataFrame after adding 'CME_ActivityID' column and dropping rows with missing values:


,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [32]:
# Convert the 'CME_ActivityID' column to string format 
gst_df['CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format 
gst_df['gstID'] = gst_df['gstID'].astype(str)

# Convert startTime to datetime format  
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])

# Rename startTime to startTime_GST 
gst_df.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
gst_df = gst_df.drop(columns=['linkedEvents'])

# Verify that all steps were executed correctly
print("\nGST DataFrame after formatting columns and renaming:")
display(gst_df.head())


GST DataFrame after formatting columns and renaming:


,gstID,startTime_GST,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01 01:00:00+00:00,2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T02:47:00-MPC-001


In [33]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.
cme_related_gsts = gst_df[gst_df['CME_ActivityID'].str.contains('CME', na=False)]


print("\nFiltered DataFrame with CME-related GSTs only:")
display(cme_related_gsts.head())


Filtered DataFrame with CME-related GSTs only:


,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [53]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(
    cme_related_gsts,  # GST DataFrame
    expanded_cme_df,   # CME DataFrame
    left_on=['gstID', 'CME_ActivityID'],      # Columns from the GST DataFrame
    right_on=['GST_ActivityID', 'cmeID'],     # Columns from the CME DataFrame
    suffixes=('_gst', '_cme')
)

# Display the merged DataFrame in a neat table format
print("\nMerged DataFrame:")
from IPython.display import display
display(merged_df.head())




Merged DataFrame:


,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


In [54]:
# Verify that the new DataFrame has the same number of rows as cme and gst
print("\nRow count comparison:")
print("Rows in merged DataFrame:", merged_df.shape[0])
print("Rows in GST DataFrame:", gst_df.shape[0])
print("Rows in CME DataFrame:", expanded_cme_df.shape[0])


Row count comparison:
Rows in merged DataFrame: 61
Rows in GST DataFrame: 205
Rows in CME DataFrame: 1714


### Computing the time it takes for a CME to cause a GST

In [55]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = (merged_df['startTime_GST'] - merged_df['startTime_CME']).dt.total_seconds() / 3600  # Convert to hours


print("\nMerged DataFrame with timeDiff:")
display(merged_df[['gstID', 'CME_ActivityID', 'startTime_GST', 'startTime_CME', 'timeDiff']].head(10))  # Displaying first 10 rows


Merged DataFrame with timeDiff:


,gstID,CME_ActivityID,startTime_GST,startTime_CME,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-02T20:24:00-CME-001,2013-06-07 03:00:00+00:00,2013-06-02 20:24:00+00:00,102.600000
1,2013-10-02T03:00:00-GST-001,2013-09-29T22:40:00-CME-001,2013-10-02 03:00:00+00:00,2013-09-29 22:40:00+00:00,52.333333
2,2013-12-08T00:00:00-GST-001,2013-12-04T23:12:00-CME-001,2013-12-08 00:00:00+00:00,2013-12-04 23:12:00+00:00,72.800000
3,2014-02-19T03:00:00-GST-001,2014-02-16T14:15:00-CME-001,2014-02-19 03:00:00+00:00,2014-02-16 14:15:00+00:00,60.750000
4,2014-02-20T03:00:00-GST-001,2014-02-18T01:25:00-CME-001,2014-02-20 03:00:00+00:00,2014-02-18 01:25:00+00:00,49.583333
5,2014-02-27T18:00:00-GST-001,2014-02-25T01:25:00-CME-001,2014-02-27 18:00:00+00:00,2014-02-25 01:25:00+00:00,64.583333
6,2014-06-08T03:00:00-GST-001,2014-06-04T13:25:00-CME-001,2014-06-08 03:00:00+00:00,2014-06-04 13:25:00+00:00,85.583333
7,2014-08-19T21:00:00-GST-001,2014-08-15T21:12:00-CME-001,2014-08-19 21:00:00+00:00,2014-08-15 21:12:00+00:00,95.800000
8,2014-09-12T15:00:00-GST-001,2014-09-10T18:18:00-CME-001,2014-09-12 15:00:00+00:00,2014-09-10 18:18:00+00:00,44.700000
9,2015-03-17T06:00:00-GST-001,2015-03-15T02:00:00-CME-001,2015-03-17 06:00:00+00:00,2015-03-15 02:00:00+00:00,52.000000


In [56]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
time_stats = merged_df['timeDiff'].describe()
print("\nTime difference statistics:")
display(time_stats)


Time difference statistics:


count     61.000000
mean      71.490710
std       23.885927
min       32.600000
25%       52.000000
50%       67.850000
75%       85.583333
max      147.000000
Name: timeDiff, dtype: float64

### Exporting data in csv format

In [57]:
# Export data to CSV without the index
merged_df.to_csv('merged_cme_gst_data.csv', index=False)
print("\nData exported to 'merged_cme_gst_data.csv'")


Data exported to 'merged_cme_gst_data.csv'
